In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeasture
import matplotlib.pyplot as plt
import xarray as xr

import metpy.calc as mpcalc
from metpy.testing import get_test_data

In [3]:
# Open netCDF file as a xarray Dataset
data = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))

#view a summary of the Dataset
print(data)

C:\Anaconda3\lib\site-packages\pooch\core.py:232: UserWarning: Downloading data file 'irma_gfs_example.nc' from remote data store 'https://github.com/Unidata/MetPy/raw/v0.9.1/staticdata/' to 'C:\Users\kan.dai\AppData\Local\metpy\cache\v0.9.1'.
  action, fname, self.base_url, str(self.path)


<xarray.Dataset>
Dimensions:                              (isobaric1: 21, isobaric3: 31, latitude: 81, longitude: 131, time1: 9)
Coordinates:
  * time1                                (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:00:00
    reftime                              datetime64[ns] ...
  * latitude                             (latitude) float32 50.0 49.5 ... 10.0
  * isobaric3                            (isobaric3) float64 100.0 ... 1e+05
  * isobaric1                            (isobaric1) float64 1e+04 ... 1e+05
  * longitude                            (longitude) float32 250.0 ... 315.0
Data variables:
    Vertical_velocity_pressure_isobaric  (time1, isobaric1, latitude, longitude) float32 ...
    Relative_humidity_isobaric           (time1, isobaric3, latitude, longitude) float32 ...
    Temperature_isobaric                 (time1, isobaric3, latitude, longitude) float32 ...
    u-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) floa

In [4]:
# Parse the dataset for projection and coordinate information
data = data.metpy.parse_cf()

In [6]:
# Parse only a single variable
data_var = data.metpy.parse_cf('Temperature_isobaric')

In [7]:
data_var

<xarray.DataArray 'Temperature_isobaric' (time1: 9, isobaric3: 31, latitude: 81, longitude: 131)>
[2960469 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:00:00
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * latitude   (latitude) float32 50.0 49.5 49.0 48.5 ... 11.5 11.0 10.5 10.0
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
    crs        object Projection: latitude_longitude
  * isobaric3  (isobaric3) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
Attributes:
    long_name:                      Temperature @ Isobaric surface
    units:                          K
    Grib_Variable_Id:               VAR_0-0-0_L100
    Grib2_Parameter:                [0 0 0]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Temperature
    Grib2_Parameter_Name:           Temperature
    Grib2_Level_Type:               100
    Grib2_Level_Desc:

In [8]:
data.rename({
    'Vertical_velocity_pressure_isobaric': 'omega',
    'Relative_humidity_isobaric': 'relative_humidity',
    'Temperature_isobaric': 'temperature',
    'u-component_of_wind_isobaric': 'u',
    'v-component_of_wind_isobaric': 'v',
    'Geopotential_height_isobaric': 'height'
}, inplace=True)

<xarray.Dataset>
Dimensions:                       (isobaric1: 21, isobaric3: 31, latitude: 81, longitude: 131, time1: 9)
Coordinates:
  * time1                         (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:00:00
    reftime                       datetime64[ns] 2017-09-05T12:00:00
  * latitude                      (latitude) float32 50.0 49.5 ... 10.5 10.0
  * isobaric1                     (isobaric1) float64 1e+04 1.5e+04 ... 1e+05
  * longitude                     (longitude) float32 250.0 250.5 ... 315.0
    crs                           object Projection: latitude_longitude
  * isobaric3                     (isobaric3) float64 100.0 200.0 ... 1e+05
Data variables:
    omega                         (time1, isobaric1, latitude, longitude) float32 ...
    relative_humidity             (time1, isobaric3, latitude, longitude) float32 ...
    temperature                   (time1, isobaric3, latitude, longitude) float32 ...
    u                             (time1, 

In [9]:
data['isobaric1'].metpy.convert_units('hPa')

In [11]:
data['isobaric3'].metpy.convert_units('hPa')

In [12]:
x, y = data['temperature'].metpy.coordinates('x', 'y')

In [14]:
vertical, = data['temperature'].metpy.coordinates('vertical')

In [15]:
time = data['temperature'].metpy.time

In [16]:
print([coord.name for coord in (x,y,vertical,time)])

['longitude', 'latitude', 'isobaric3', 'time1']


In [17]:
data_crs = data['temperature'].metpy.cartopy_crs

In [20]:
data_globe = data['temperature'].metpy.cartopy_globe

In [21]:
lat, lon = xr.broadcast(y, x)

In [22]:
f = mpcalc.coriolis_parameter(lat)